In [1]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np


In [3]:
PATH = Path.cwd().parent.joinpath('data', 'voi')

In [4]:
# Import vessel data

dfs = []

for file in PATH.joinpath('gfw_data', 'vessel_info').glob('*.csv'):
    name = file.stem.split('-')[0]
    dfs.append(pd.read_csv(file))

df = pd.concat(dfs)
len(df)


232

In [5]:
# Clean ownership data

owner = df[['id', 'owner']].copy()
owner.owner = owner.owner.str.split('|')
owner = owner.explode('owner')
owner['start_date'] = owner.owner.apply(lambda x: x.split(' (')[1] if x is not np.nan else x)
owner['end_date'] = owner.start_date.apply(lambda x: x.split('Z-')[1] if x is not np.nan else x)
owner.start_date = owner.start_date.apply(lambda x: pd.to_datetime(x[0:10]) if x is not np.nan else x)
owner.end_date = owner.end_date.apply(lambda x: pd.to_datetime(x[0:10]) if x is not np.nan else x)
owner.owner = owner.owner.apply(lambda x: x.split(' (')[0].split('-')[0] if x is not np.nan else x)
owner.head()

,id,owner,start_date,end_date
0,afdc56c195a6d87d098b54fff09cda96,TAVR,2018-09-19,2024-02-29
0,afdc56c195a6d87d098b54fff09cda96,MENRON,2018-09-19,2024-02-29
0,c02d4df5d65d5b36f8ba62703fb2da05,NORTHERN SHIPPING,2014-01-01,2023-01-18
0,69dc5cb5a-a5db-0ca2-11c0-f6d7c6b3572e,NaN,NaT,NaT
0,a7986785254313f9976b4509c152ca21,VOLGA SHIPPING JOINT STOCK,2014-01-01,2024-02-29


In [6]:
owner.groupby(owner.end_date.dt.year).agg(aantal_bedrijven=('owner', lambda x: x.nunique()))

,aantal_bedrijven
end_date,
2016.0,2
2017.0,7
2018.0,3
2019.0,2
2020.0,2
2021.0,7
2022.0,6
2023.0,31
2024.0,105


In [7]:
owner.groupby('id').owner.count().sort_values(ascending=False).nlargest(10)

id
0937cfe77a5a003794abac7f4bed1666    7
dde5c260378aa0728258667e6bf0cc8d    5
38bb7eb7ecaa8961aa06a209ed3f70ca    5
012b1605fb79de2bd760a3cffbde3840    4
f286a02f534c16a1bc14ba274d77d756    4
d8c5381fe5f7141be889cb856dff86cf    4
d69db440341cc98aed61631c2244c07a    4
41e9b7938527f5f06661562efcede268    4
c6cae58a7ac8d235cddc64f703484856    4
c3e22e0e3dc28668dc9dbcc4aab1de79    4
Name: owner, dtype: int64

In [10]:
owner[owner.start_date > '2022-02-22'].owner.value_counts()

owner
MG FLOT                     5
ANTEY                       2
EGOR LETOV SHIPPING         2
JSC GTLK                    1
SCF ECO                     1
IDAN SHIPPING               1
TRANSMORFLOT                1
INVESTNEFTETRADE            1
TRAFFER AIRCRAFT LEASING    1
FARVATER                    1
ESPERANTO                   1
Name: count, dtype: int64